## Welcome!

This guide will give you some sense of the features and resources in the PagerDuty API, using the `pagerduty` Python library.

To use this guide, you will need:
* A PagerDuty account, with access to at least one [service](https://support.pagerduty.com/main/docs/service-profile), [team](https://support.pagerduty.com/main/docs/teams), [escalation policy](https://support.pagerduty.com/main/docs/escalation-policies), and [oncall schedule](https://support.pagerduty.com/main/docs/schedule-basics). If you don't have a PagerDuty account, you can sign up for a [trial](https://www.pagerduty.com/sign-up/) or a [developer account](https://developer.pagerduty.com/sign-up/), depending on your goals.
* An [API key](https://support.pagerduty.com/main/docs/api-access-keys) to access your PagerDuty account via the API.
* A working Python3 environment.

### Get Started

Install the `pagerduty` Python package in your environment.
If you are using Anaconda, activate your envrionment first, or create a new environment for your PagerDuty work.

`pip install pagerduty`

The `pagerduty` Python package is maintained by folks at PagerDuty. You can find the project at [Github](https://github.com/PagerDuty/python-pagerduty) and the docs in the [user guide](https://pagerduty.github.io/python-pagerduty/user_guide.html).


In [ ]:
!pip install pagerduty

In [ ]:
import pagerduty as pd

### Session and Keys

To work with the PagerDuty REST API you'll need an **API Access Key**.  If you've never created a key on your PagerDuty account, please read the documentation [here](https://support.pagerduty.com/main/docs/api-access-keys).

There are two types of API Access Keys:
* *General Access*: These keys are account-global, and can access all objects in a PagerDuty account. They must be created by an account Admin or Account Owner.
* *User Token*: These keys are linked to a specific user login, and will have access only to the objects that user account has access to. Any requests the user isn't permitted to make will have a `403 Forbidden` response.
* *OAuth tokens*: These are generally for integrations and applications, see the [docs](https://developer.pagerduty.com/docs/register-an-app) for more info on how to set up your custom integration.


Initialize the session and authentication.

In [ ]:
api_token = "ADD YOUR KEY HERE"

session = pd.RestApiV2Client(api_token)

### European Service Region
<p style="background:yellow;color:black">If your account is in the EU region, uncomment the next line:</p>

In [ ]:
# client.url = "https://api.eu.pagerduty.com

### Make Your First Request - Get a Service

First, we are going to request a service, which will give us some other object IDs to investigate. The API endpoint `/services` has a number of options, but we're going to use `limit=1` to just show one service.

When we access the data in the results, we'll still need to account for the data being in an array.

For more on the `/services` endpoint, see the documentation [here](https://developer.pagerduty.com/api-reference/e960cca205c0f-list-services).

In [ ]:
results = session.rget('/services?limit=1')

my_service = results[0]
my_service

You can see the full structure of a single service.


> When we use the pagerduty library's `rget` method, it will take care of serializing the payload into an object that we can reference, without having to explicitly load the JSON string. You can immediately access the keys using `object["key"]` notation. You can also the `get` method or the `jget` method. See the docs for more details. If you use a general purpose HTTP library, you'll need to load the JSON into an addressable object yourself.


In the payload are some important things to look out for:

* `id`: every object in the PagerDuty REST API has an ID associated with it. These are short alphanumeric strings, many of which will start with a capital `P`.
* `name`: The human-readable name of the object.
* `description`, or, sometimes `summary`: The description entered for this object.
* `type`: the object type of the object. These will often point you to what other objects you can query to gather more information.

Let's use this first service as a start to look at other interesting and related objects. We can parse out some pieces from the JSON and use those.

In [ ]:
service_id = my_service['id']
service_id

### Get a Team and its Members

First, let's look at `/teams`. The `teams` key in our service object is a top-level key, and has a sub-key `id` that we can use to get more information from the API about the team that owns this service.

In the `service` object, the `teams` key also contains an array, though you might not see any services with more than one owning team. When working with the PagerDuty REST API, keep an eye on the structure of the data returned, since there will be many parts that can have multiple values!

In [ ]:
my_team = my_service['teams'][0]['id']
my_team

Many of the objects in the PagerDuty REST API have two ways to get basic information - the main `/objects` endpoint, which will return a list of all the objects in the account: `/incidents`, `/services`, `/teams`, etc.

If you know the `id` of an individual object, you can often use the same endpoint to request just that instance - `/incidents/{id}`, `/services/{id}`, `/teams/{id}`. You'll see these two options listed in the documentation for the objects that support them.


For more info on the `/teams` endpoint, see the documentation [here](https://developer.pagerduty.com/api-reference/1c8181e57cc60-get-a-team).

In [ ]:
team_object = session.rget(f"/teams/{my_team}")
team_object

In these results, we can see the summary data of the team that owns the service returned above. If we want to see the members of that team, we need to add `/members` to the endpoint:

In [ ]:
team_members = session.rget(f"/teams/{my_team}/members")
team_members

<div class="alert alert-block alert-info">
Note that every user has their own unique `id` in PagerDuty, just like any other object!
</div>

Other interesting things in the team members objects:
* `self`: you'll see this on every return, so you can access an individual object that might be contained in the results.
* `html_url`: the URL in the web UI for this object. Helpful if you are referencing pages back and forth from UI to API.

Now we have two ways we can request objects from the original `service` JSON object - we can access the `id`s of other objects included in the results, and we can use the `self` urls.

### Get an Escalation Policy

Let's get the `escalation_policy` associated with the original `service`.  See the `escalation_policy` [docs](https://developer.pagerduty.com/api-reference/3db5a206585e1-get-an-escalation-policy) for more details on Escalation Policy actions.

In [ ]:
my_escalation_policy_id = my_service['escalation_policy']['id']
my_escalation_policy_id

In [ ]:
my_escalation_policy_object = session.rget(f"/escalation_policies/{my_escalation_policy_id}")
my_escalation_policy_object

We can see from this output that an `escalation_policy` is a fairly large compound object - it's comprised of a number of other objects that represent the responders included in the escalation policy as well as the rules that govern how they should be notified.

Depending on how the escalation policy is configured, you might see some sub-objects like users, defined by a `user_reference` or an on-call schedule, defined as a `schedule_reference`. When you are looking at compound objects and you're not sure exactly what is in the data, look for the `type` entries to give you an idea of what other information is included, and what other objects you can investigate.

### Get an On Call Schedule

If your escalation policy includes a `schedule_reference`, you can run the next block. If not, skip to the section on **Incidents**.

In [ ]:
# find a schedule in the escalation_rules
rules = my_escalation_policy_object['escalation_rules']

for rule in rules:
    for target in rule['targets']:
        if target['type'] == "schedule_reference":
            sched_id = target['id']
            break
sched_id

Schedules in PagerDuty are reusable objects that can be included in multiple escalation policies. Information in a schedule includes the responders included in the schedule and the rules that govern the way the shifts are created. To get more information about a schedule, we can use the `/schedules/` [endpoint](https://developer.pagerduty.com/api-reference/3f03afb2c84a4-get-a-schedule).

If your escalation policy above included a `schedule`, run the next code block:

In [ ]:
schedule = session.rget(f"/schedules/{sched_id}")
schedule


## Incidents

Incidents are the main component you'll likely be working with in your PagerDuty account. Events come into PagerDuty, are translated into `alerts`, and many then become `incidents`. Incidents then start the workflows that include `escalation_policies` and `schedules`.

You can list, get, and update incidents via the API, as well as manage the alerts assigned to an incident, add responders and subscribers, snooze an incident, and a number of other actions. See the [docs](https://developer.pagerduty.com/api-reference/9d0b4b12e36f9-list-incidents) for more details.

We'll look at a few simple operations to give you a feel for the incident object. As with the objects we saw above, the payload of an incident object will be a JSON document, and we'll find a number of other objects referenced within it.

To run the next section of the notebook, you should have some incidents in your account. They do not have to be active, but you can create a new incident to work with if you'd like.

In [ ]:
my_incident = session.rget("/incidents?limit=1")
my_incident

You can see an `incident` is a fairly large object, containing references to a number of other objects. Some of the items to note:

* `id`: Every `incident` has an `id` which you can use to access it directly via the API.
* `title`, `summary`, and `description`: May be entered by the monitoring software that sent the original alerts, or by the person who created the incident. Will be passed on to responders in their notifications.
* `status`: Incidents can be `triggered`, `acknowledged`, or `resolved`.
* `service`: The `service` the incident is assigned to. As we saw above, `escalation_policies` are assigned to `services`. It's through the `service` assignment that an `incident` will trigger an `escalation_policy`. An `incident` itself can't trigger the policy, it must be assigned to a `service`.
* `assignments`: This is where you can find the responders who have been assigned to an incident, either via the `escalation_policy` or manually. You'll see the `escalation_policy` listed as well, and the `teams` that own the service.
* `first_trigger_log_entry` is an interesting item. Each `incident` has a log of actions, like updates, that can be accessed in the web UI in the "Timeline" tab. You can see the first log entry here. There is another endpoint for collection the other log entries.
* `priority` is another object type. Each `priority` in your account will have an `id` associated with it, because these items are configurable by the account owners.


Once you have the `id` of an individual incident, there are a number of other actions you can take.

* List all of the alerts associated with an `incident`. If your `service` has alert grouping enabled, there could be many alerts associated with a single `incident`. See more on alert grouping in the [docs](https://support.pagerduty.com/main/docs/alert-grouping).
* See the `incident`'s relationship to any Business Services configured on the account. More on Business Services [here](https://support.pagerduty.com/main/docs/business-services).
* Create or list `notes` for the `incident`. These are often manually added, and are visible in the web UI in the "Notes" box on the right side of the screen. These are useful for documenting what actions were taken to resolve an incident.
* Get any related or past `incidents` that might be helpful in resolving the incident.
* Create a `status_update` for the incident. Status updates might be sent to a status page, an internal chat, or to subscribers. They are helpful for keeping non-responders informed about what's going on with the `incident`. Check out the [knowledge base page](https://support.pagerduty.com/main/docs/communicate-with-stakeholders#status-update-notification-methods) for more details.

## Writing to the PagerDuty API

So far we've only sent a set of simple `GET` requests to the API, but many folks will want to update objects or create new ones via the API as well. These are also done with the `pagerduty` library, and the data structures will also be in JSON.

Each object will have different **required** and **optional** keys in its payload. The [docs](https://developer.pagerduty.com/api-reference/) will help you figure out which pieces have to be included and which can be included if you need them.

The API docs also have sample code you can use when creating your requests, so you'll have a guide for creating your payloads.

## Example: Create a New Service

You can run the following cells to create a new service on your PagerDuty account to get a feel for how the requests work. We'll use the components from the original service to help us bootstrap the payload for this request.

There are two methods each for `PUT`s and `POST`s from the `pagerduty` library, similar to the `GET` methods, for creating fully wrapped objects to send to the API. I'll use `rpost` here, but you can check the docs for how to use `post`.

In [ ]:
payload = {
    "service": {
        "type": "service",
        "name": "Test Service from Jupyter Notebook",
        "description": "My cool web application that does things.",
        "auto_resolve_timeout": 14400,
        "acknowledgement_timeout": 600,
        "status": "active",
        "escalation_policy": {
            "id": f"{my_escalation_policy_id}",
            "type": "escalation_policy_reference"
        }
    }
}
payload


Now we have the payload, we'll create the request:

In [ ]:
response = session.rpost('services', json=payload)
response

That's all there is to it! You've created a service.

If you'd like to delete it, run the following cell:

In [ ]:
new_service_id = response['id']
delete_response = session.delete(f"/services/{new_service_id}")
delete_response

## Other Things to Look Out For

### Pagination
When you've been using PagerDuty for a while, you'll collect a large number of some objects, especially incidents, but also maybe services and other components, if your organization is large. There are limits to how many objects can be returned at a time, so many of the endpoints in the PagerDuty REST API support pagination.

<div class="alert alert-block alert-info">
By default, responses are batched into sets of 25 results. You can change that up to 100.

With pagination, the full response set is limited to 10000 records. If you have a set that results in more than 10000 records, you'll want to take a look at filtering those results or talk to the Support team, who may be able to get a larger data dump for you.
</div>

Some endpoints will have pagination anchors returned in their response payloads. You will see `limit`, `offset`, `more`, and `total` included in these responses.

Other endpoints support cursor based pagination. Instead of the four page anchors, you'll see `limit` and `next_cursor` in the response.

The `pagerduty` Python library also includes iterators to help you traverse a full set of records, including `iter_all` and `list_all` methods.

See the [pagination documentation](https://developer.pagerduty.com/docs/pagination) for more on using pagination.

### Filtering
Some endpoints support filtering, allowing you to request a subset of the data available. These filters behave more like a `contains` search, and do not support regular expressions. You can access these filter capabilities with the `name` parameter on the objects that support it.

Some filters are more explicit and are listed in the documentation for an endpoint. There are a number of endpoints that allow you to filter on the `id`s of one or more objects, like a team or user. These filters will have brackets in their parameter names, which can be confusing.

```
team_services = session.rget('services', params={'team_ids[]: ["teamID"]'})
```

## Learn More

* API [Documentation](https://developer.pagerduty.com/docs/introduction)
* REST API [Endpoint References](https://developer.pagerduty.com/api-reference/f1a95bb9397ba-changelog)
* [Support Knowledge Base](https://support.pagerduty.com)
* [User Guide](https://pagerduty.github.io/python-pagerduty/user_guide.html) for the `pagerduty` Python package
* Join the [PagerDuty Commons](https://community.pagerduty.com) community